In [54]:
# constant variables
DEFAULT_PRICE_DIGIT = 2;
DEFAULT_SETTLE_DIGIT = 2;
PERCENT_CHANGE_DIGIT = 2;
PERCENT_SWING_DIGIT = 2;
PERFECT_PROJECTED_CHANGE_DIGIT = 2;
INOUT_DIGIT = 2;
EQUITY_PREVIOUS_CLOSE_DIGIT = 2;
PE_DIGIT = 2;
PBV_DIGIT = 2;
YIELD_DIGIT = 2;
EPS_DIGIT = 2;
W52_DIGIT = 2;
PERCENT_CHANGE_WEEK_MONTH_DIGIT = 2;
LAST_TRADING_DATE_LENGTH = 10;
PERCENT_DIGIT = 2;
DATE_LENGTH = 8;
ORDER_SIDE_LENGTH = 1;
DERIVATIVES_WARRANT_TYPE_LENGTH = 1;
EXERCISE_DATE_LENGTH = 10;
W_52_HIGH_LOW_DATE_LENGTH = 10;
AVERAGE_DIGIT = 2;
WARRANT_UNDERLYING_PRICE_DIGIT = 2;
INAV_DIGIT = 4;
TIME_VALUE_DIGIT = 2;
TIME_LENGTH = 8;
POSITION_LENGTH = 1;
UNTIL_LENGTH = 8;
OPTIONS_WIZARD_TOKEN_DIGIT = 2;
TRADE_DATE_CHART_LENGTH = 6;
AUM_SIZE_DIGIT = 2;
THEORICAL_PRICE_DIGIT = 2;
IMPLIED_VOLATILLITY_DIGIT = 2;
DELTA_DIGIT = 2;
THETA_DIGIT = 4;
HISTORICAL_CHART_VALUE_DIGIT = 2;
NEWS_DATE_LENGTH = 10;
NEWS_TIME_LENGTH = 8;
INITIAL_MARGIN_DIGIT = 2;
NOTIFICATION_DATE_LENGTH = 10;
NOTIFICATION_TIME_LENGTH = 5;
STRIKE_PRICE_DIGIT = 2;
HISTORICAL_BY_DATE_VALUE_DIGIT = 2;
MULTIPLER_DIGIT = 5;
EXPIRY_TIME_LENGTH = 5;
EXPIRY_DATE_LENGTH = 10;


In [56]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import sys;
import time;
import json;
from selenium import webdriver;
import selenium
import selenium.webdriver.support.ui as UI
import re;
import socket
import urllib
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import datetime
import hashlib
import re
from struct import *

#----- Configuration

config={}
config['username'] = 'crawler001'
config['password'] = 'crawler001'
config['url']      = 'https://click2win.settrade.com'
config['APIVersion']      = '1'
config['hourshift']      = 15 # from server to thai timezone +7 

# ----- Start crawling 
path='/Users/nik9618/Desktop/Library/chromedriver';
driver = webdriver.Chrome(path)
driver.get(config['url'])
print 'Main page loaded'
sys.stdout.flush()
time.sleep(1);

# ----- Login
userBox = driver.find_element_by_name('txtLogin')
passBox = driver.find_element_by_name('txtPassword')
userBox.clear()
passBox.clear()
userBox.send_keys(config['username'])
passBox.send_keys(config['password'])
elem = driver.find_element_by_class_name('input_login_submit-login');
elem.click();
print 'Logged In'
sys.stdout.flush()
time.sleep(3);


Main page loaded
Logged In


In [29]:

# ------ Generate flash variables
driver.get('view-source:'+config['url']+'/realtime/streaming5/flash/StreamingPage.jsp')
# print 'view-source:'+conf['url']+'/realtime/streaming5/flash/StreamingPage.jsp'
print 'Streaming Opened'
sys.stdout.flush()
time.sleep(1)
src = driver.page_source
src = src[src.find('flashVar'):]
src = src[src.find('{')+1:]
src = src[0:src.find('}')]
flashVar = {}
src = src.split(",")
for line in src:
	line = line.split(":")
	flashVar[urllib.unquote(line[0])] = urllib.unquote(line[1][1:-1])
print 'Flash Variable Generated'
sys.stdout.flush()
time.sleep(1);

Streaming Opened
Flash Variable Generated


In [4]:
driver.get(config['url'] + flashVar['fvSyncTimeURL'])
time = driver.find_element_by_tag_name('body').text.split("\r")[0].split("|")

thaiTime = int(time[1]) + config['hourshift']*60*60*1000
thaiTime = thaiTime/1000

print 'time synchronized'
print time


time synchronized
[u'T', u'1450078079351', u'1450078079351']


In [5]:
url = config['url'] + flashVar['fvGenerateKeyURL']
key = datetime.datetime.fromtimestamp(thaiTime).strftime('%d/%m/%Y') + "_" + flashVar['fvBrokerId']+"_"+flashVar['fvUserref']
m = hashlib.md5()
m.update(key)
hs = m.hexdigest()
print "hash("+key +") = " + hs
script = ""
script = script + 'sc=\'\'; sc += \'<form id="dynForm" action="'+url+'" method="post">\';'
script = script + 'sc += \'<input type="hidden" name="time" value="'+time[1]+'">\';'
script = script + 'sc += \'<input type="hidden" name="clientType" value="'+flashVar['fvRealtimeClientType']+'">\';'
script = script + 'sc += \'<input type="hidden" name="txtSETNET3" value="'+flashVar['fvSETNET3']+'">\';'
script = script + 'sc += \'<input type="hidden" name="APIVersion" value="'+config['APIVersion']+'">\';'
script = script + 'sc += \'<input type="hidden" name="q" value="'+hs+'">\';'
script = script + 'sc += \'</form>\';'
script = script + 'document.body.innerHTML += sc;'
# print script;

script = script + 'document.getElementById("dynForm").submit()';
# driver.get('https://click2win.settrade.com'+flashVar['fvGenerateKeyURL'])
# 
driver.execute_script(script)
src = driver.find_element_by_tag_name('body').text.split("\r")[0].split("|")
print src

hash(14/12/2015_089_7575997) = 74d14e67ddf571ff7f8c7e68d8286c89
[u'T', u'D90YW3tubDqonTettjObug==', u'pushctw1.settrade.com', u'3000']


In [6]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
dns = socket.gethostbyname_ex(src[2]);
host = dns[2][0];
port = int(src[3])
conn = s.connect((host, port))

# print conn
print src[2]
# print dns
print host
print port
s.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)

pushctw1.settrade.com
202.60.196.165
3000


In [7]:
ss = flashVar['fvUserref']+"|"+flashVar['fvBrokerId']+"|"+src[1]+"|" + str(int(time[1])+3313) + "|gen5sc1|1\n"
print s.send(ss)
print ss;

61
7575997|089|D90YW3tubDqonTettjObug==|1450078082664|gen5sc1|1



In [8]:
# print s.send("REG|5\n")
# print s.send("REG|4^N~N^N~N^E~D^E~D\n")

print s.send("REG|1^AOT~AAV~NOK~THAI~MINT~SUSCO~KTC~DTAC~INTUCH~TKN^^M~M~M~M~M~M~M~M~M~M\n")

# print s.send("REG|1^^^M\n")
# print s.send("REG|3^^\n")
# print s.send("REG|5\n")
# print s.send("REG|4^^N~N^^E~D\n")
# print s.send("REG|1^^^M\n")
# print s.send("REG|3^^\n")
# print s.send("REG|5\n")
# print s.send("REG|3^^\n")


75


In [ ]:
# service 5 >>> market summary
service = unpack('>B',msg[0])[0];
config = unpack('>B',msg[1])[0];
flag = unpack('>B',msg[2])[0];

print "service : "+ str(service);
print "config : "+ str(config);
print "flag : "+ str(flag);

pos=3
total = {}
read=4; div=2; total['setIndex'] = unpack('>I',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
read=4; div=2; total['setHigh'] = unpack('>I',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
read=4; div=2; total['setLow'] = unpack('>I',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;

read=2; div=2; total['setChange'] = unpack('>h',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
read=2; div=2; total['setHighChange'] = unpack('>h',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
read=2; div=2; total['setLowChange'] = unpack('>h',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;

read=4; div=2; total['setTotalValue'] = unpack('>I',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
read=8; div=0; total['setTotalVolume'] = unpack('>Q',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;

read=2; div=0; total['setGainers'] = unpack('>H',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
read=2; div=0; total['setLosers'] = unpack('>H',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
read=2; div=0; total['setUnchanged'] = unpack('>H',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;

read=1; div=0; size = unpack('>B',msg[pos:pos+read])[0]; pos+=read;
read=size; div=0; total['setStatus'] = ''.join(msg[pos:pos+read]); pos+=read;

print total


In [9]:
# service x >>> parseInfo :: bids offers
size = s.recv(2)
size = unpack('>H',size)[0]
msg = s.recv(size)

    

In [13]:
# REQUIREMENT
# - it has to be an instrument i.e. no .ARGI

while(True):
    size = s.recv(2)
    size = unpack('>H',size)[0]
    msg = s.recv(size)

    service = unpack('>B',msg[0])[0];
    configByte = unpack('>B',msg[1])[0];

    conf = {};
    conf['iosV'] = (configByte>>3) & 1
    conf['loi'] = (configByte>>2) & 1
    conf['ios'] = (configByte>>1) & 1
    conf['sob'] = (configByte>>0) & 1

    print conf;
    pos=2;

    def readIOSV(conf,m,p,d=0,unsigned=True):
        if(unsigned):
            if(conf['iosV'] == 1):
                pos=p; read=2; div=d; ans = unpack('>H',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
            else:
                pos=p; read=4; div=d; ans = unpack('>I',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
        else:
            if(conf['iosV'] == 1):
                pos=p; read=2; div=d; ans = unpack('>h',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
            else:
                pos=p; read=4; div=d; ans = unpack('>i',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
        if(d==0):
            return (pos,int(ans))
        else:
            return (pos,ans)


    def readLOI(conf,m,p,d=0,unsigned=True):
        if(unsigned):
            if(conf['loi'] == 1):
                pos=p; read=8; div=d; ans = unpack('>Q',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
            else:
                pos=p; read=4; div=d; ans = unpack('>I',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
        else:
            if(conf['loi'] == 1):
                pos=p; read=8; div=d; ans = unpack('>q',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
            else:
                pos=p; read=4; div=d; ans = unpack('>i',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
        if(d==0):
            return (pos,int(ans))
        else:
            return (pos,ans)

    def readIOS(conf,m,p,d=0,unsigned=True):
        if(unsigned):
            if(conf['ios'] == 1):
                pos=p; read=4; div=d; ans = unpack('>I',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
            else:
                pos=p; read=2; div=d; ans = unpack('>H',msg[pos:pos+read])[0]/pow(10.,div); pos+=read; 
        else:
            if(conf['ios'] == 1):
                pos=p; read=4; div=d; ans = unpack('>i',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
            else:
                pos=p; read=2; div=d; ans = unpack('>h',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
        if(d==0):
            return (pos,int(ans))
        else:
            return (pos,ans)

    def readSOB(conf,m,p,d=0,unsigned=True):
        if(unsigned):
            if(conf['sob'] == 1):
                pos=p; read=2; div=d; ans = unpack('>H',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
            else:
                pos=p; read=1; div=d; ans = unpack('>B',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
        else:
            if(conf['sob'] == 1):
                pos=p; read=2; div=d; ans = unpack('>h',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
            else:
                pos=p; read=1; div=d; ans = unpack('>b',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
        if(d==0):
            return (pos,int(ans))
        else:
            return (pos,ans)

    def readDouble(m,p):
        pos=p; read=8; ans = unpack('>d',msg[pos:pos+read])[0]; pos+=read;
        return (pos,ans)

    def readInt(m,p,d):
        pos=p; read=4;div=d; ans = unpack('>i',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
        if(d==0):
            return (pos,int(ans))
        else:
            return (pos,ans)

    def readByte(m,p,unsigned=True):
        if(unsigned):
            pos=p; read=1; ans = unpack('>B',msg[pos:pos+read])[0]; pos+=read;
        else:
            pos=p; read=1; ans = unpack('>b',msg[pos:pos+read])[0]; pos+=read;
        return (pos,ans)

    def readString(m,p,size):
        pos=p; read=size; div=0; ans = ''.join(msg[pos:pos+read]); pos+=read;
        return (pos,ans)


    (pos,size) = readByte(msg,pos);
    (pos,instrument) = readString(msg,pos,size);
    (pos,instrumentType) = readByte(msg,pos);
    print instrument
    
    if(service==4 and instrument == 'AOT'):
        pass;
    else:
        continue;
    
    instSubtype = -1
    if(instrumentType==0): # 0 = equity
        (pos,instSubType) = readByte(msg,pos,unsigned=False);

    priceDigit = DEFAULT_PRICE_DIGIT;
    settleDigit = DEFAULT_SETTLE_DIGIT;

    if (instrumentType==1 or instrumentType==2): # 1 = futures 2 = options
        (pos,priceDigit) = readByte(msg,pos);
        (pos,settleDigit) = readByte(msg,pos);

    (pos,flag) = readByte(msg,pos);

    hasInitMarket = (flag >> 5) & 1;
    hasInitIntraday = (flag >> 4) & 1;
    hasInitStat = (flag >> 3) & 1;
    hasSummary = (flag >> 2) & 1;
    hasBidOffer = (flag >> 1) & 1;
    hasProjected = (flag >> 0) & 1;

    print hasInitMarket
    print hasInitIntraday
    print hasInitStat
    print hasSummary
    print hasBidOffer
    print hasProjected

    if(hasInitMarket == 1):
        (pos,previousClose) = readIOS(conf,msg,pos,priceDigit);
        (pos,high) = readIOS(conf,msg,pos,priceDigit);
        (pos,low) = readIOS(conf,msg,pos,priceDigit);
        (pos,ceiling) = readIOS(conf,msg,pos,priceDigit);
        (pos,floor) = readIOS(conf,msg,pos,priceDigit);
        if (instrumentType==1 or instrumentType==2): #--- is derivatives
            spread = readIOS(conf,msg,pos,priceDigit);
            previousSettle = readIOS(conf,msg,pos,priceDigit);

    if(hasInitIntraday == 1):
        if (instrumentType ==0 or instrumentType ==1 or instrumentType ==2):
            (pos,size) = readSOB(conf,msg,pos);
            (pos,longInstName) = readString(msg,pos,size);
            print longInstName

            if(instrumentType==0): # equity
                pos,pe = readIOS(conf,msg,pos,PE_DIGIT);
                pos,pbv = readIOS(conf,msg,pos,PBV_DIGIT);
                pos,yield_ = readIOS(conf,msg,pos,YIELD_DIGIT);
                pos,eps = readIOS(conf,msg,pos,EPS_DIGIT);
                pos,w52High = readIOS(conf,msg,pos,W52_DIGIT);
                pos,w52Low = readIOS(conf,msg,pos,W52_DIGIT);
                pos,w52HighDate = readString(msg,pos,W_52_HIGH_LOW_DATE_LENGTH); #--------- string
                pos,w52LowDate = readString(msg,pos,W_52_HIGH_LOW_DATE_LENGTH); #--------- string
                pos,percentChg1W = readIOS(conf,msg,pos,PERCENT_CHANGE_WEEK_MONTH_DIGIT);
                pos,percentChg1M = readIOS(conf,msg,pos,PERCENT_CHANGE_WEEK_MONTH_DIGIT);
                pos,percentChg3M = readIOS(conf,msg,pos,PERCENT_CHANGE_WEEK_MONTH_DIGIT);
                pos,size = readByte(msg,pos);
                pos,currency = readString(msg,pos,size);
                if (instSubtype==2 or instSubtype==3): #W and DW
                    pos,exercisePrice = readDouble(msg,pos);
                    pos,exerciseDate = readString(msg,pos,EXERCISE_DATE_LENGTH);
                    pos,isCallOptions = readByte(msg,pos);
                    if(isCallOption>0):
                        optionsType = 'CALL';
                    else:
                        optionsType = 'PUT';
                    pos,size = readByte(msg,pos);
                    pos,exerciseRatio = readString(msg,pos,size);

                    if (instSubtype==3): #DW
                        pos,derivativesWarrantType = readString(msg,pos,DERIVATIVES_WARRANT_TYPE_LENGTH);
                        pos,remainingDays = readIOS(conf,msg,pos);
                        pos,lastTradingDate = readString(msg,pos,LAST_TRADING_DATE_LENGTH);
                        pos,multiplier = readInt(msg,pos,MULTIPLER_DIGIT);
                else:
                    if (instSubtype == 4):
                        pos,aumSize = readInt(msg,pos,AUM_SIZE_DIGIT);
            else:
                if(instrumentType==1 or instrumentType==2 ): # futures and options 
                    pos,lifeHigh = readIOS(conf,msg,pos,W52_DIGIT);
                    pos,lifeLow = readIOS(conf,msg,pos,W52_DIGIT);
                    pos,lifeHighDate = readString(msg,pos,W_52_HIGH_LOW_DATE_LENGTH);
                    pos,lifeLowDate = readString(msg,pos,W_52_HIGH_LOW_DATE_LENGTH);
                    pos,remainingDays = readIOS(conf,msg,pos);
                    pos,lastTradingDate = readString(msg,pos,LAST_TRADING_DATE_LENGTH);
                    pos,size = readByte(msg,pos);
                    pos,underlying = readString(msg,pos,size);
                    pos,size = readByte(msg,pos);
                    pos,markettype = readString(msg,pos,size);
                    pos,initialMargin = readInt(msg,pos,INITIAL_MARGIN_DIGIT);
                    if (instrumentType==2):
                        pos,isCallOptions = readByte(msg,pos);
                        if(isCallOptions>0):
                            optionsType = 'CALL';
                        else:
                            optionsType = 'PUT';
                        pos,strikePrice = readIOS(conf,msg,pos,STRIKE_PRICE_DIGIT);
        else:
            #read index
            pos,marketCap = readLOI(conf,msg,pos);
            pos,pe = readIOS(conf,msg,pos,PE_DIGIT);
            pos,pbv = readIOS(conf,msg,pos,PBV_DIGIT);
            pos,yield_ = readIOS(conf,msg,pos,YIELD_DIGIT);

    if(hasInitStat==1):
        if (instrumentType==0):
            pos,dividendYield = readIOS(conf,msg,pos)

    if(hasSummary==1):
        pos,lastDone = readIOS(conf,msg,pos,priceDigit);
        pos,change = readIOS(conf,msg,pos,priceDigit);
        pos,percentChange = readIOS(conf,msg,pos,PERCENT_DIGIT);
        pos,totalVolume = readLOI(conf,msg,pos);
        if (instrumentType ==0 or instrumentType ==1 or instrumentType ==2):
            pos,average = readIOS(conf,msg,pos,priceDigit);
            pos,size = readByte(msg,pos);
            pos,status = readString(msg,pos,size);
            pos,percentBuy = readByte(msg,pos);
            pos,percentSell = readByte(msg,pos);
            if (instrumentType==0):
                pos,totalValue = readLOI(conf,msg,pos)
                pos,averageBuy = readIOS(conf,msg,pos,AVERAGE_DIGIT);
                pos,averageSell = readIOS(conf,msg,pos,AVERAGE_DIGIT);
                if (instSubtype==2 or instSubtype==3): 
                    pos,underlyingPrice = readIOS(conf,msg,pos,WARRANT_UNDERLYING_PRICE_DIGIT);
                    pos,theoricalPrice = readIOS(conf,msg,pos,THEORICAL_PRICE_DIGIT);
                    pos,impliedVolatility = readIOS(conf,msg,pos,IMPLIED_VOLATILLITY_DIGIT);
                    pos,delta = readIOS(conf,msg,pos,DELTA_DIGIT);
                    pos,theta = readInt(msg,pos,THETA_DIGIT);
                else:
                    if (instSubtype==4): #etf
                        pos,iNav = readInt(msg,pos,INAV_DIGIT);
            else:
                if (instrumentType ==1 or instrumentType ==2):
                    pos,underlyingPrice = readIOS(conf,msg,pos,priceDigit);
                    pos,openInterest = readIOS(conf,msg,pos);
                    pos,settlePrice = readIOS(conf,msg,pos,settleDigit);
                    pos,theoricalPrice = readIOS(conf,msg,pos,THEORICAL_PRICE_DIGIT);
                    if (instrumentType ==2):
                        pos,impliedVolatility = readIOS(conf,msg,pos,IMPLIED_VOLATILLITY_DIGIT);
                        pos,delta = readIOS(conf,msg,pos,DELTA_DIGIT);
                        pos,theta = readInt(msg,pos,THETA_DIGIT);
        else:
            if (instrumentType==3):
                pos,totalValue = readLOI(conf,msg,pos)

    if (hasBidOffer==1):
        if (instrumentType ==0 or instrumentType ==1 or instrumentType ==2):
            pos,bidAskFlagArray = readByte(msg,pos);
            bidFlag = bidAskFlagArray & 0x0F;
            askFlag = bidAskFlagArray >> 4;
            pos,bidPrice1 = readIOS(conf,msg,pos,priceDigit);
            pos,bidPrice2 = readIOS(conf,msg,pos,priceDigit);
            pos,bidPrice3 = readIOS(conf,msg,pos,priceDigit);
            pos,bidPrice4 = readIOS(conf,msg,pos,priceDigit);
            pos,bidPrice5 = readIOS(conf,msg,pos,priceDigit);
            pos,askPrice1 = readIOS(conf,msg,pos,priceDigit);
            pos,askPrice2 = readIOS(conf,msg,pos,priceDigit);
            pos,askPrice3 = readIOS(conf,msg,pos,priceDigit);
            pos,askPrice4 = readIOS(conf,msg,pos,priceDigit);
            pos,askPrice5 = readIOS(conf,msg,pos,priceDigit);
            pos,bidVolume1 = readLOI(conf,msg,pos);
            pos,bidVolume2 = readLOI(conf,msg,pos);
            pos,bidVolume3 = readLOI(conf,msg,pos);
            pos,bidVolume4 = readLOI(conf,msg,pos);
            pos,bidVolume5 = readLOI(conf,msg,pos);
            pos,askVolume1 = readLOI(conf,msg,pos);
            pos,askVolume2 = readLOI(conf,msg,pos);
            pos,askVolume3 = readLOI(conf,msg,pos);
            pos,askVolume4 = readLOI(conf,msg,pos);
            pos,askVolume5 = readLOI(conf,msg,pos);

            print "\t"+str(bidFlag) + "\t" + str(askFlag)
            print str(bidVolume1) +"\t"+str(bidPrice1)  + "\t" + str(askPrice1)  + "\t" + str(askVolume1) 
            print str(bidVolume2) +"\t"+str(bidPrice2)  + "\t" + str(askPrice2)  + "\t" + str(askVolume2) 
            print str(bidVolume3) +"\t"+str(bidPrice3)  + "\t" + str(askPrice3)  + "\t" + str(askVolume3) 
            print str(bidVolume4) +"\t"+str(bidPrice4)  + "\t" + str(askPrice4)  + "\t" + str(askVolume4) 
            print str(bidVolume5) +"\t"+str(bidPrice5)  + "\t" + str(askPrice5)  + "\t" + str(askVolume5) 

    print lastDone
    print change
    print percentChange
    print totalVolume


{'sob': 0, 'loi': 0, 'ios': 1, 'iosV': 0}
AOT
1
0
0
1
1
1
	2	2
11900	0.0	0.0	11300
800	330.0	310.0	1900
400	329.0	311.0	100
1100	324.0	316.0	2100
100	321.0	318.0	100
320.0
42949665.96
42949670.82
1401200


In [ ]:
# service 4 >>> ticker parsing
while(True):
    size = s.recv(2)
    size = unpack('>H',size)[0]
    msg = s.recv(size)
    #print msg
    service = unpack('>B',msg[0])[0];
    config = unpack('>B',msg[1])[0];
    intinstrumentType = unpack('>B',msg[2])[0];
    
    print service
    
    signature = unpack('>B',msg[3])[0];

    tickerSubType = (signature) & 0x0F
    orderSide = (signature>>4) & 0x01
    trend = (signature>>5) & 0x03
    isSum = (signature>>7) & 0x01

    iosV = config>>3 & 1
    loi = config>>2 & 1
    ios = config>>1 & 1
    sob = config>>0 & 1

    if(tickerSubType == 0): tickerSubType='Common'
    elif(tickerSubType == 1): tickerSubType='Foriegn'
    elif(tickerSubType == 2): tickerSubType='ETF'
    elif(tickerSubType == 3): tickerSubType='DW'
    elif(tickerSubType == 4): tickerSubType='W'
    elif(tickerSubType == 5): tickerSubType='Convertible'
    elif(tickerSubType == 6): tickerSubType='Preferred'
    elif(tickerSubType == 7): tickerSubType='UnitTrust'

    if(orderSide == 0):
        orderSide='Buy'
    elif(orderSide == 1):
        orderSide='Sell'
    elif(orderSide == 2):
        orderSide='Short'
    elif(orderSide == 3):
        orderSide='Cover'

    if(trend == 0):
        trend='Up'
    elif(trend == 1):
        trend='-'
    elif(trend == 2):
        trend='Down'
    elif(trend == 3):
        trend='NoData'

    if(intinstrumentType==0): # equity
        priceDigit = 2;
        pos=4
    else:
        priceDigit = unpack('>B',msg[4])[0];
        pos=5
        
    total = {}
    read=1; div=0; size = unpack('>B',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
    size=int(size)
    read=size; div=0; instrument = ''.join(msg[pos:pos+read]); pos+=read;

    print pos
    print len(msg)
    if(ios == 0):
        read=2; div=2; price = unpack('>H',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
        read=2; div=2; change = unpack('>h',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
    else:
        read=4; div=2; price = unpack('>I',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;
        read=4; div=2; change = unpack('>i',msg[pos:pos+read])[0]/pow(10.,div); pos+=read;

    read=4; div=0; seqID = int(unpack('>I',msg[pos:pos+read])[0]/pow(10.,div)); pos+=read;

    if(sob==0):
        read=1; div=0; volCount = int(unpack('>B',msg[pos:pos+read])[0]/pow(10.,div)); pos+=read;
    else:
        read=2; div=0; volCount = int(unpack('>H',msg[pos:pos+read])[0]/pow(10.,div)); pos+=read;

    volumns=[]
    for i in range(volCount):
        if(iosV==0):
            read=2; div=0; volumn = int(unpack('>H',msg[pos:pos+read])[0]/pow(10.,div)); pos+=read;
        else:
            read=4; div=0; volumn = int(unpack('>I',msg[pos:pos+read])[0]/pow(10.,div)); pos+=read;
        volumns.append(volumn);

    # print "tickerType= " + tickerSubType
    # print "orderSide= " + orderSide
    # print "trend= " + trend
    # print "isSum= " + str(isSum)
    print str(seqID)+"\t"+str(instrument)+"\t"+str(price)+"\t" + str(change) + "\t"+str(volCount)+"\t"+ str(volumns)
    sys.stdout.flush();

In [84]:
url = config['url'] + flashVar['fvDataProviderStrURL']
key = datetime.datetime.fromtimestamp(thaiTime).strftime('%d/%m/%Y') + "_" + flashVar['fvBrokerId']+"_"+flashVar['fvUserref']
m = hashlib.md5()
m.update(key)
hs = m.hexdigest()
print "hash("+key +") = " + hs
script = ""
script = script + 'sc=\'\'; sc += \'<form id="dynForm" action="'+url+'" method="post">\';'
script = script + 'sc += \'<input type="hidden" name="boardType" value="'+"equity"+'">\';'
script = script + 'sc += \'<input type="hidden" name="APIVersion" value="'+config['APIVersion']+'">\';'
script = script + 'sc += \'<input type="hidden" name="subListName" value="">\';'
script = script + 'sc += \'<input type="hidden" name="mainListName" value=".A">\';'
script = script + 'sc += \'<input type="hidden" name="boardSubType" value="">\';'
script = script + 'sc += \'<input type="hidden" name="service" value="12">\';'
script = script + 'sc += \'<input type="hidden" name="q" value="'+hs+'">\';'
script = script + 'sc += \'</form>\';'
script = script + 'document.body.innerHTML += sc;'
# print script;

script = script + 'document.getElementById("dynForm").submit()';
# driver.get('https://click2win.settrade.com'+flashVar['fvGenerateKeyURL'])
# 
driver.execute_script(script)
print url
text = driver.find_element_by_tag_name('body').text.split("\r")[0].split("|")

print len(text)
if text[0] == 'T':
    category = text[6].split("^")
    print category
    futures = [];
    options = [];
    equity  = [];
    indexes = [];
    for cate in category:
        if cate == 'futuresAndUnderlying':
            insts = text[7].split("^")
            for inst in insts:
                inst = inst.split("~")[0]
                futures.append(inst);
                
        if cate == 'options':
            insts = text[8].split("^")
            for inst in insts:
                inst = inst.split("~")[0]
                options.append(inst);
                
        if cate == 'equity':
            insts = text[9].split("^")
            for inst in insts:
                inst = inst.split("~")[0]
                equity.append(inst);
                
    new_equity =[]
    
    reW = re.compile('^.*-W\d(\d*)$')
    reDW = re.compile('^(.|..|...|....)\d\d(C|P)\d\d\d\d.$')
    reF = re.compile('^.*-F$')
    reP = re.compile('^.*-P$')
    reQ = re.compile('^.*-Q$')
      
    for eq in equity:
        if(reW.match(eq) or reDW.match(eq) or reF.match(eq) or reP.match(eq) or reQ.match(eq)):
            #not simple
            pass;
        else:
            new_equity.append(eq);
    
    equity = new_equity;
    
    print len(futures)
    print len(options)
    print len(equity)
    total = len(futures)+ len(options) + len(equity)
    print total

hash(14/12/2015_089_7575997) = 74d14e67ddf571ff7f8c7e68d8286c89
https://click2win.settrade.com/realtime/streaming5/Streaming5DataProvider.jsp
16
[u'futuresAndUnderlying', u'options', u'equity', u'index', u'industry', u'maiIndustry', u'sector']
564
82
716
1362
